# Localization

In [191]:
import open3d as o3d
import pandas as pd
import copy
import os
import numpy as np
import time

#load ground truth csc as pandas dataframe
g_df = pd.read_csv("/app/dataset/ground_truth.csv")

#load input frames (source)
frames_dir = "/app/dataset/frames"
frame_files = os.listdir(frames_dir)
#frames = [o3d.io.read_point_cloud(os.path.join(frames_dir, f)) for f in frame_files]

#load target map


## ICP Localization ##

In [118]:

start_time = time.time()



initTransform = np.identity(4)
for i, frame in enumerate(frames):
    # Downsample the point cloud using a voxel grid filter


    frame_cloud_downsampled = frame.voxel_down_sample(voxel_size = 0.1)



    source = copy.deepcopy(frame_cloud_downsampled)
    target = copy.deepcopy(map_cloud)

    # Filter points with z-axis values less than 1
    #source = source.transform(initTransform)
    
    points = np.asarray(source.points)
# Apply the transformation matrix to the point

#To transform a 3D points array using a 4x4 transformation matrix in Python, you can use the NumPy library. Here's an example code that shows how to do it:




    # Transform the 3D points by multiplying them with the transformation matrix
    # Add an extra row of 1s to the points array to make it homogeneous
    homogeneous_points = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed_homogeneous_points = np.dot(initTransform, homogeneous_points.T).T

    # Divide the x, y, and z coordinates by the last column to remove the homogeneous coordinate
    transformed_points = transformed_homogeneous_points[:, :3] / transformed_homogeneous_points[:, 3:]




    filtered_points = transformed_points[transformed_points[:, 2] > 1]
    # Convert back to open3d point cloud
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

    #trans_init = np.identity(4)
    threshold = 1
 
    reg_result = o3d.pipelines.registration.registration_icp(filtered_pcd, target, threshold)

    initTransform = reg_result.transformation * initTransform


    
    frame_gt = g_df[g_df['Frame'] == i]

    
# Calculate L2 norm 
    errors = np.linalg.norm(np.array([initTransform[0,3],initTransform[1,3],initTransform[2,3]])-np.array([ frame_gt[' x'].item(),  frame_gt[' y'].item(),  frame_gt[' z'].item()]))   

    print('Frame :', i)
    print("Errors--------:", errors)


end_time = time.time()
total_time = end_time - start_time

print(f"Total execution time: {total_time:.3f} seconds")

### Load the map from the map.pcd file

In [119]:
map_cloud = o3d.io.read_point_cloud("/app/dataset/map.pcd") #projects/dataset/map.pcd

# Print the number of points in the map cloud
print(f"Loaded {len(map_cloud.points)} data points from map.pcd")

Loaded 143949 data points from map.pcd


### Iteritatively go through each frame and localize the car in the given map.

In [80]:
import os
import numpy as np
import time

In [81]:
frames_dir = "/app/dataset/frames"
frame_files = os.listdir(frames_dir)
frames = [o3d.io.read_point_cloud(os.path.join(frames_dir, f)) for f in frame_files[:500]]

KeyboardInterrupt: 

In [ ]:
print(frames[0].get_center())

[-0.29560711 -0.48462959 -0.79555747]


In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])

In [ ]:
threshold = 0.02
trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
                         [-0.139, 0.967, -0.215, 0.7],
                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
#draw_registration_result(map_cloud, frames[0], trans_init)  #does not work, because i forgot to start Xlauncher first, lol

In [ ]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    map_cloud, frames[0], threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=6.946905e-06, inlier_rmse=2.677878e-03, and correspondence_set size of 1
Access transformation to get result.


In [ ]:
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    map_cloud, frames[0], threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
#draw_registration_result(map_cloud, frames[0], reg_p2p.transformation)

Apply point-to-point ICP
RegistrationResult with fitness=6.946905e-06, inlier_rmse=0.000000e+00, and correspondence_set size of 1
Access transformation to get result.
Transformation is:
[[ 0.862       0.011      -0.507       0.50254853]
 [-0.139       0.967      -0.215       0.69950987]
 [ 0.487       0.255       0.835      -1.39933986]
 [ 0.          0.          0.          1.        ]]


In [ ]:
voxel_size = 0.2
icp_params = o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100)
icp_params.relative_fitness = 1e-6
icp_params.relative_rmse = 1e-6

In [ ]:
# try it on 1 frame
frame_cloud = frames[0].voxel_down_sample( voxel_size = 0.2)

In [ ]:
frame.get_center()

array([ 0.07772482, -1.00245112, -0.7023499 ])

In [ ]:
class Pose:
    def __init__(self, point, rotate):
        self.point = point
        self.rotate = rotate

class Point:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

class Rotate:
    def __init__(self, roll, pitch, yaw):
        self.roll = roll
        self.pitch = pitch
        self.yaw = yaw

def getPose(matrix):
    pose_point = Point(matrix[0,3], matrix[1,3], matrix[2,3])
    pose_rotate = Rotate(np.arctan2(matrix[1, 0], matrix[0, 0]), np.arctan2(-matrix[2,0], np.sqrt(matrix[2,1]**2 + matrix[2,2]**2)), np.arctan2(matrix[2,1], matrix[2,2]))
    pose = Pose(pose_point, pose_rotate)
    return pose


def transform3D(yaw, pitch, roll, xt, yt, zt):
    matrix = np.identity(4)

    matrix[0, 3] = xt
    matrix[1, 3] = yt
    matrix[2, 3] = zt

    matrix[0, 0] = np.cos(yaw) * np.cos(pitch)
    matrix[0, 1] = np.cos(yaw) * np.sin(pitch) * np.sin(roll) - np.sin(yaw) * np.cos(roll)
    matrix[0, 2] = np.cos(yaw) * np.sin(pitch) * np.cos(roll) + np.sin(yaw) * np.sin(roll)
    matrix[1, 0] = np.sin(yaw) * np.cos(pitch)
    matrix[1, 1] = np.sin(yaw) * np.sin(pitch) * np.sin(roll) + np.cos(yaw) * np.cos(roll)
    matrix[1, 2] = np.sin(yaw) * np.sin(pitch) * np.cos(roll) - np.cos(yaw) * np.sin(roll)
    matrix[2, 0] = -np.sin(pitch)
    matrix[2, 1] = np.cos(pitch) * np.sin(roll)
    matrix[2, 2] = np.cos(pitch) * np.cos(roll)

    return matrix

In [ ]:

def transform_vector_to_affine(x):
    # Extract translation and rotation values
    tx, ty, tz, roll, pitch, yaw = x

    # Create rotation matrix from roll, pitch, yaw angles
    cr = np.cos(roll)
    sr = np.sin(roll)
    cp = np.cos(pitch)
    sp = np.sin(pitch)
    cy = np.cos(yaw)
    sy = np.sin(yaw)

    rotation_matrix = np.array([
        [cp*cy, cp*sy, -sp],
        [sr*sp*cy - cr*sy, sr*sp*sy + cr*cy, sr*cp],
        [cr*sp*cy + sr*sy, cr*sp*sy - sr*cy, cr*cp]
    ])

    # Create translation vector
    translation_vector = np.array([tx, ty, tz])

    # Combine rotation and translation to create affine transformation matrix
    affine_transform = np.eye(4)
    affine_transform[:3, :3] = rotation_matrix
    affine_transform[:3, 3] = translation_vector

    return affine_transform

def rmse(T1, T2):
    # Compute RMSE between two transformation matrices
    diff = T1 - T2
    sq_err = np.multiply(diff, diff)
    mse = np.mean(sq_err)
    return np.sqrt(mse)


In [84]:
transf = frame_cloud_downsampled.transform(initTransform)
points = np.asarray(transf.points)

In [186]:
    points = np.asarray(map_cloud.points)
    filtered_points = points[points[:, 2] > 1]
    # Convert back to open3d point cloud
    map_cloud = o3d.geometry.PointCloud()
    map_cloud.points = o3d.utility.Vector3dVector(filtered_points)

In [187]:
map_cloud

PointCloud with 29196 points.

In [190]:
start_time = time.time()



initTransform = np.identity(4)
for i, frame in enumerate(frames):
    # Downsample the point cloud using a voxel grid filter


    frame_cloud_downsampled = frame.voxel_down_sample(voxel_size = 0.1)



    source = copy.deepcopy(frame_cloud_downsampled)
    target = copy.deepcopy(map_cloud)

    # Filter points with z-axis values less than 1
    #source = source.transform(initTransform)
    
    points = np.asarray(source.points)
# Apply the transformation matrix to the point

#To transform a 3D points array using a 4x4 transformation matrix in Python, you can use the NumPy library. Here's an example code that shows how to do it:




    # Transform the 3D points by multiplying them with the transformation matrix
    # Add an extra row of 1s to the points array to make it homogeneous
    homogeneous_points = np.hstack([points, np.ones((points.shape[0], 1))])
    transformed_homogeneous_points = np.dot(initTransform, homogeneous_points.T).T

    # Divide the x, y, and z coordinates by the last column to remove the homogeneous coordinate
    transformed_points = transformed_homogeneous_points[:, :3] / transformed_homogeneous_points[:, 3:]




    filtered_points = transformed_points[transformed_points[:, 2] > 1]
    # Convert back to open3d point cloud
    filtered_pcd = o3d.geometry.PointCloud()
    filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)

    #trans_init = np.identity(4)
    threshold = 1
 
    reg_result = o3d.pipelines.registration.registration_icp(filtered_pcd, target, threshold)

    initTransform = reg_result.transformation * initTransform


    
    frame_gt = g_df[g_df['Frame'] == i]

    
# Calculate L2 norm 
    errors = np.linalg.norm(np.array([initTransform[0,3],initTransform[1,3],initTransform[2,3]])-np.array([ frame_gt[' x'].item(),  frame_gt[' y'].item(),  frame_gt[' z'].item()]))   

    print('Frame :', i)
    print("Errors--------:", errors)


end_time = time.time()
total_time = end_time - start_time

print(f"Total execution time: {total_time:.3f} seconds")

Frame : 0
Errors--------: 0.0
Frame : 1
Errors--------: 0.0156357
Frame : 2
Errors--------: 0.00584579
Frame : 3
Errors--------: 0.00562843
Frame : 4
Errors--------: 0.00562843
Frame : 5
Errors--------: 0.00562843
Frame : 6
Errors--------: 0.00562843
Frame : 7
Errors--------: 0.00562843
Frame : 8
Errors--------: 0.00562843
Frame : 9
Errors--------: 0.00562843
Frame : 10
Errors--------: 0.00562843
Frame : 11
Errors--------: 0.00562843
Frame : 12
Errors--------: 0.00591316
Frame : 13
Errors--------: 0.00677673
Frame : 14
Errors--------: 0.00704376
Frame : 15
Errors--------: 0.00710373
Frame : 16
Errors--------: 0.007099
Frame : 17
Errors--------: 0.00697754
Frame : 18
Errors--------: 0.00698051
Frame : 19
Errors--------: 0.00700378
Frame : 20
Errors--------: 0.00701149
Frame : 21
Errors--------: 0.00707542
Frame : 22
Errors--------: 0.00705162
Frame : 23
Errors--------: 0.00704933
Frame : 24
Errors--------: 0.00705543
Frame : 25
Errors--------: 0.0608321398169042
Frame : 26
Errors-------

KeyboardInterrupt: 

In [ ]:

#To transform a 3D points array using a 4x4 transformation matrix in Python, you can use the NumPy library. Here's an example code that shows how to do it:


import numpy as np


# Transform the 3D points by multiplying them with the transformation matrix
# Add an extra row of 1s to the points array to make it homogeneous
homogeneous_points = np.hstack([points, np.ones((points.shape[0], 1))])
transformed_homogeneous_points = np.dot(initTransform, homogeneous_points.T).T

# Divide the x, y, and z coordinates by the last column to remove the homogeneous coordinate
transformed_points = transformed_homogeneous_points[:, :3] / transformed_homogeneous_points[:, 3:]

# Print the transformed points
print(transformed_points)

[[ -1.610835   -3.4847713  -1.7901864]
 [ -1.7770556  -3.4030092  -1.7901864]
 [ -1.8586328  -3.3591485  -1.7901864]
 ...
 [  4.3966212 -21.229404   -1.6509889]
 [ -2.7634912  -6.3023229  -1.6377864]
 [ -2.6106298  -6.3671579  -1.6377864]]


In [183]:

#To transform a 3D points array using a 4x4 transformation matrix in Python, you can use the NumPy library. Here's an example code that shows how to do it:


import numpy as np


# Transform the 3D points by multiplying them with the transformation matrix
# Add an extra row of 1s to the points array to make it homogeneous
homogeneous_points = np.hstack([points, np.ones((points.shape[0], 1))])
transformed_homogeneous_points = np.dot(initTransform, homogeneous_points.T).T

# Divide the x, y, and z coordinates by the last column to remove the homogeneous coordinate
transformed_points = transformed_homogeneous_points[:, :3] / transformed_homogeneous_points[:, 3:]

# Print the transformed points
print(transformed_points)

[[ -1.610835   -3.4847713  -1.7901864]
 [ -1.7770556  -3.4030092  -1.7901864]
 [ -1.8586328  -3.3591485  -1.7901864]
 ...
 [  4.3966212 -21.229404   -1.6509889]
 [ -2.7634912  -6.3023229  -1.6377864]
 [ -2.6106298  -6.3671579  -1.6377864]]


In [155]:
((initTransform[:3,:3] @ points.T).reshape(-1,3))

(9268, 3)

array([[0., 0., 0.]])

In [158]:
(initTransform[:3,:3] @ points.T).reshape(-1,3) +   initTransform[:3,3:].T

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

### Use performance measures to evaluate the localization effectiviness.

Remember that to pass this task, the maximum lateral error cannot be bigger than 1.2 meters.

### Evaluate the computing time (there is not a harsh requirement here, but should be analized.)